- How do we handle multiple sequences?
- How do we handle multiple sequences of different lengths?
- Are vocabulary indices the only inputs that allow a model to work well?
- Is there such a thing as too long a sequence?

Let’s see what kinds of problems these questions pose, and how we can solve them using the 🤗 Transformers API.

## Models expect a batch of inputs

In the previous exercise you saw how sequences get translated into lists of numbers. Let’s convert this list of numbers to a tensor and send it to the model:

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a Hugging Face course my whole life."

tokens = tokenizer.tokenize(sequence)
print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

input_ids = torch.tensor(ids)
print(input_ids)

model(input_ids)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['I', "'", 've', 'been', 'waiting', 'for', 'a', 'Hu', '##gging', 'Face', 'course', 'my', 'whole', 'life', '.']
[146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 10289, 1736, 1139, 2006, 1297, 119]
tensor([  146,   112,  1396,  1151,  2613,  1111,   170, 20164, 10932, 10289,
         1736,  1139,  2006,  1297,   119])


IndexError: too many indices for tensor of dimension 1

Oh no! Why did this fail? We followed the steps from the pipeline in section 2.

> The problem is that we sent a single sequence to the model, whereas 🤗 Transformers models expect multiple sentences by default. Here we tried to do everything the tokenizer did behind the scenes when we applied it to a sequence. But if you look closely, you’ll see that the tokenizer didn’t just convert the list of input IDs into a tensor, it added a dimension on top of it:

In [8]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,   146,   112,  1396,  1151,  2613,  1111,   170, 20164, 10932,
         10289,  1736,  1139,  2006,  1297,   119,   102]])


Let’s try again and add a new dimension:

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a Hugging Face course my whole life."

tokens = tokenizer.tokenize(sequence)
print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

input_ids = torch.tensor([ids]) ##Added extra dimension to the tensor
print(input_ids)

model(input_ids)


['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', 'face', 'course', 'my', 'whole', 'life', '.']
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 2227, 2607, 2026, 2878, 2166, 1012]
tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,  2227,  2607,
          2026,  2878,  2166,  1012]])


SequenceClassifierOutput(loss=None, logits=tensor([[-3.5496,  3.7656]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

> Batching is the act of sending multiple sentences through the model, all at once. If you only have one sentence, you can just build a batch with a single sequence:

In [13]:
batched_ids = [ids, ids] #This is a batch of two identical sequences!

In [15]:
batched_input_ids = torch.tensor(batched_ids)
print(batched_input_ids)

model(batched_input_ids)


tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,  2227,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,  2227,  2607,
          2026,  2878,  2166,  1012]])


SequenceClassifierOutput(loss=None, logits=tensor([[-3.5496,  3.7656],
        [-3.5496,  3.7656]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Batching allows the model to work when you feed it multiple sentences. Using multiple sequences is just as simple as building a batch with a single sequence. There’s a second issue, though. When you’re trying to batch together two (or more) sentences, they might be of different lengths. If you’ve ever worked with tensors before, you know that they need to be of rectangular shape, so you won’t be able to convert the list of input IDs into a tensor directly. To work around this problem, we usually pad the inputs.

## Padding the inputs

The following list of lists cannot be converted to a tensor:

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

In order to work around this, we’ll use padding to make our tensors have a rectangular shape. Padding makes sure all our sentences have the same length by adding a special word called the padding token to the sentences with fewer values. For example, if you have 10 sentences with 10 words and 1 sentence with 20 words, padding will ensure all the sentences have 20 words. In our example, the resulting tensor looks like this:

In [ ]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

The padding token ID can be found in tokenizer.pad_token_id. Let’s use it and send our two sentences through the model individually and batched together:

In [16]:
token_pad = tokenizer.pad_token_id
print(token_pad)

0


In [17]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


There’s something wrong with the logits in our batched predictions: the second row should be the same as the logits for the second sentence, but we’ve got completely different values!

> This is because the key feature of Transformer models is attention layers that contextualize each token. These will take into account the padding tokens since they attend to all of the tokens of a sequence. To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask.

## Attention masks

Attention masks are tensors with the exact same shape as the input IDs tensor, filled with 0s and 1s: 1s indicate the corresponding tokens should be attended to, and 0s indicate the corresponding tokens should not be attended to (i.e., they should be ignored by the attention layers of the model).

Let’s complete the previous example with an attention mask:

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [20]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
sequence1 = "I've been waiting for a Hugging Face course my whole life."
sequence2 = "I hate this so much!"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [sequence1, sequence2]

sequence1_tokens = tokenizer.tokenize(sequence1)
sequence2_tokens = tokenizer.tokenize(sequence2)

print(sequence1_tokens)
print(sequence2_tokens)

sequence1_ids = tokenizer.convert_tokens_to_ids(sequence1_tokens)
sequence2_ids = tokenizer.convert_tokens_to_ids(sequence2_tokens)

print(sequence1_ids)
print(sequence2_ids)

output1 = model(torch.tensor([sequence1_ids]))
output2 = model(torch.tensor([sequence2_ids]))

print(output1.logits)
print(output2.logits)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', 'face', 'course', 'my', 'whole', 'life', '.']
['i', 'hate', 'this', 'so', 'much', '!']
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 2227, 2607, 2026, 2878, 2166, 1012]
[1045, 5223, 2023, 2061, 2172, 999]
tensor([[-3.5496,  3.7656]], grad_fn=<AddmmBackward0>)
tensor([[ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)


In [23]:
sequences = ["I've been waiting for a Hugging Face course my whole life.", "I hate this so much!"]

sequences_tokens = [tokenizer.tokenize(seq) for seq in sequences]
print(sequences_tokens)

sequences_ids = [tokenizer.convert_tokens_to_ids(seq) for seq in sequences_tokens]
print(sequences_ids)

from torch.nn.utils.rnn import pad_sequence

# Convert lists of ids to tensors
sequences_ids_tensors = [torch.tensor(ids) for ids in sequences_ids]
# Pad to the same length
padded_ids = pad_sequence(sequences_ids_tensors, batch_first=True, padding_value=tokenizer.pad_token_id)
# Adjust attention_mask accordingly


attention_mask = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(padded_ids, attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

[['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', 'face', 'course', 'my', 'whole', 'life', '.'], ['i', 'hate', 'this', 'so', 'much', '!']]
[[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 2227, 2607, 2026, 2878, 2166, 1012], [1045, 5223, 2023, 2061, 2172, 999]]
tensor([[-3.5496,  3.7656],
        [ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)


## Longer sequences

With Transformer models, there is a limit to the lengths of the sequences we can pass the models. Most models handle sequences of up to 512 or 1024 tokens, and will crash when asked to process longer sequences. There are two solutions to this problem:

Use a model with a longer supported sequence length.
Truncate your sequences.
Models have different supported sequence lengths, and some specialize in handling very long sequences. Longformer is one example, and another is LED. If you’re working on a task that requires very long sequences, we recommend you take a look at those models.

Otherwise, we recommend you truncate your sequences by specifying the max_sequence_length parameter:

In [ ]:
sequence = sequence[:max_sequence_length]